<a href="https://colab.research.google.com/github/tatsath/Interpretability/blob/main/CreditRiskExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This cookbook provides some examples of how to use features and steering in Goodfire in sometimes non-traditional ways.

Such as:
- Dynamic prompts
- Removing Knowledge
- Sorting by features
- On-demand RAG


In [ ]:
!pip install goodfire --quiet
!pip install datasets --quiet


In [ ]:
from google.colab import userdata

# Add your Goodfire API Key to your Colab secrets
GOODFIRE_API_KEY = userdata.get('GOODFIRE_API_KEY')

## Initialize the SDK

In [ ]:
import goodfire

client = goodfire.Client(GOODFIRE_API_KEY)

# Instantiate a model variant
#variant = goodfire.Variant("meta-llama/Meta-Llama-3.1-8B-Instruct")
variant = goodfire.Variant("meta-llama/Llama-3.3-70B-Instruct")

## Removing knowledge

Let's say we want a model to not know anything about famous people so that we don't get in trouble if it says bad things about them.

We'll use feature search to find features that are relevant to famous people and then play with what happens.

In [ ]:
famous_people_features = client.features.search("credit risk awareness", model=variant, top_k=10)
print(famous_people_features)

FeatureGroup([
   0: "Concerns or uncertainty about credit scores and creditworthiness",
   1: "Financial credit concepts and terminology",
   2: "Technical discussions of credit risk assessment and evaluation",
   3: "Financial risk and investment caution",
   4: "Credit rating agency designations and terminology",
   5: "Discussions of credit card fraud, theft, or security risks",
   6: "Financial and business risk concepts",
   7: "The assistant is providing structured financial advice about credit scores",
   8: "Bank-related content requiring safety/security considerations",
   9: "Risk assessment and management methodology descriptions"
])


In [ ]:
# edits = client.features.AutoSteer(
#     specification="be credit risk aware",  # or your desired behavior
#     model=variant,
# )

In [ ]:
variant.set(edits)
print(edits)

FeatureEdits([
   0: (Language about obtaining loans and financial products, 0.5626923076923077)
   1: (Advice about paying off debt and debt repayment strategies, 0.2961538461538462)
   2: (Loan repayment obligations and payment schedules, 0.2961538461538462)
])


After some experimentation, we found a set of feature edits that make the model still recognize celebrity names as noteworthy individuals but forgets all personal details about them.

In [ ]:
variant.reset()
variant.set(famous_people_features[0], 0)
variant.set(famous_people_features[1], 0)

for token in client.chat.completions.create(
    [
        {"role": "user", "content": "What is the rating of this sentence on a scale of -10 to 10 :The company reported a significant drop in quarterly revenue but has successfully secured long-term financing and reduced outstanding debt."}
    ],
    model=variant,
    stream=True,
    max_completion_tokens=150,
):
    print(token.choices[0].delta.content, end="")

I'd rate this sentence a 5. It's neutral, as it reports both a negative (drop in revenue) and a positive (securing financing and reducing debt). The tone is objective and matter-of-fact, neither overly positive nor negative.

In [ ]:
variant.reset()
variant.set(famous_people_features[0], .5)
variant.set(famous_people_features[1], 0)

for token in client.chat.completions.create(
    [
        {"role": "user", "content": "What is the rating of this sentence on a scale of -10 to 10 :The company reported a significant drop in quarterly revenue but has successfully secured long-term financing and reduced outstanding debt."}
    ],
    model=variant,
    stream=True,
    max_completion_tokens=150,
):
    print(token.choices[0].delta.content, end="")

I'd rate this sentence a 6. It's a mix of both positive and negative information, but the fact that the company has secured long-term financing and reduced debt suggests a sense of stability and responsibility, which slightly outweighs the initial negative news of a drop in revenue.

## Dynamic Prompts

In this example, we'll create a model variant that responds to the user's prompt with a different response depending on whether the user is asking for code or not.

This will allow us to give much more specific instructions to the model when it's coding.

### Find Programming Features

We'll first find features that are relevant to programming. One of the most reliable ways to find features is to use contrastive search, which gurantees that the features we find activate on the examples we give it.

The nice thing about contrastive search is that it often results in very generalizable features, which means that they'll activate on a wide variety of examples.


In [ ]:
variant.reset()

_, programming_features = client.features.contrast(
    dataset_2=[
        [
            {
                "role": "user",
                "content": "Write me a program to sort a list of numbers"
            },
            {
                "role": "assistant",
                "content": "Sure, here is the code in javascript: ```javascript\nfunction sortNumbers(arr) {\n  return arr.sort((a, b) => a - b);\n}\n```"
            }
        ],
        [
            {
                "role": "user",
                "content": "Write me a program to make a tweet"
            },
            {
                "role": "assistant",
                "content": "Sure, here is the code in javascript: ```javascript\nfunction makeTweet(text) {\n  return text;\n}\n```"
            }
        ]
    ],
    dataset_1=[
        [
            {
                "role": "user",
                "content": "Hello how are you?"
            },
            {
                "role": "assistant",
                "content":
                  "I'm doing well!"
            },
        ], [
            {
                "role": "user",
                "content": "What's your favorite food?"
            },
            {
                "role": "assistant",
                "content":
                  "It's pizza!"
            },
        ]
    ],
    model=variant,
    top_k=30
)

programming_features = client.features.rerank(
    features=programming_features,
    query="programming",
    model=variant,
    top_k=5
)

print(programming_features)

# Feature # 3 is: "The user is requesting code to be written or generated"
request_programming_feature = programming_features[2]

FeatureGroup([
   0: "Specifying which programming language code should be written in",
   1: "Syntactical sugar in programming languages",
   2: "Syntactical sugar in programming languages",
   3: "The assistant is explaining how to create a new program or feature",
   4: "The assistant should complete a code snippet"
])


Next we'll use the features.inspect endpoint to check if the model is requesting code. features.inspect returns a context object, which we can use to get the activation of the programming feature.

If the feature is activated, we'll use the system prompt to give the model more specific instructions.

If the feature is not activated, we'll use the default system prompt.

Without the dynamic prompt, llama 8B tends to write less detailed code with more TODOs and fewer useful comments.

In [ ]:


def check_if_requesting_programming(prompt):
    variant.reset()
    context = client.features.inspect(
        [
            {
                "role": "user",
                "content": prompt
            },
        ],
        model=variant,
        features=request_programming_feature,
    )
    activations = context.top(k=1)
    highest_activation = max(activations, key=lambda x: x.activation)
    return highest_activation.activation > 0.5 #this threshold is arbitrary, but it's a good starting point


def generate_response(prompt):

    is_requesting_programming = check_if_requesting_programming(prompt)
    system_prompt = "You are a helpful assistant."
    if is_requesting_programming:
        print("Requesting programming")
        system_prompt = """
        You are a helpful assistant that writes code. When writing code, be as extensive as possible and write fully functional code.
        Always include comments and proper formatting.
        NEVER leave 'todos' or 'placeholders' in your code.
        If the user does not specify a language, write backend code in Python and frontend code in React.
        Do not explain what your code does, unless the user asks. Just write it.
        """

    for token in client.chat.completions.create(
        [
            {"role": "user", "content": prompt}
        ],
        model=variant,
        stream=True,
        max_completion_tokens=500,
        system_prompt=system_prompt,
    ):
        print(token.choices[0].delta.content, end="")

generate_response("Write me a program to sort a list of numbers")


**Number Sorter Program**

Below is an example of a Python program that sorts a list of numbers using the built-in `sorted` function.

```python
def sort_numbers(num_list):
    """
    Sorts a list of numbers in ascending order.

    Args:
        num_list (list): A list of numbers.

    Returns:
        list: A sorted list of numbers.
    """
    return sorted(num_list)

def main():
    # Example usage
    numbers = [64, 34, 25, 12, 22, 11, 90]
    print("Original list:", numbers)
    print("Sorted list:", sort_numbers(numbers))

if __name__ == "__main__":
    main()
```

**How it works:**

1. The `sort_numbers` function takes a list of numbers as input.
2. It uses the `sorted` function to sort the list in ascending order.
3. The sorted list is returned.
4. In the `main` function, we create an example list of numbers and print the original and sorted lists.

**Output:**
```
Original list: [64, 34, 25, 12, 22, 11, 90]
Sorted list: [11, 12, 22, 25, 34, 64, 90]
```

You can save this cod

## Sort by features

You can use feature activations as a way to filter and sort data. In this case let's find some of Elon Musk's tweets that are sarcastic.

In [ ]:
from datasets import load_dataset
num_train_samples = 100
elon_tweets = load_dataset("lcama/elon-tweets", split="train[0:100]")
elon_tweets = elon_tweets.select(range(num_train_samples))
elon_tweets


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset_infos.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

(…)-00000-of-00001-20152340fd29aa38.parquet:   0%|          | 0.00/137k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1601 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 100
})

In [ ]:
sarcasm_features = client.features.search("sarcasm in tweets", model=variant, top_k=4)
print(sarcasm_features)


FeatureGroup([
   0: "Mentions and discussions of sarcasm",
   1: "Academic or analytical discussion of sarcasm and irony",
   2: "Punctuation patterns in sarcastic or playful dialogue",
   3: "Condescending or patronizing sarcasm, especially in response to perceived basic actions"
])


Find all tweets with a sarcasm score > 1

In [ ]:
def score_sarcasm_on_tweet(tweet):
    context = client.features.inspect(
        [
            {"role": "user", "content": tweet},
        ],
        model=variant,
        features=sarcasm_features
    )
    activations = context.top(k=len(sarcasm_features))
    total_activation = sum(activation.activation for activation in activations)
    return total_activation


tweets_list = list(elon_tweets)
# get any tweets with sarcasm > 1
sarcastic_tweets = [tweet for tweet in tweets_list if score_sarcasm_on_tweet(tweet["text"]) > 1]
sarcastic_tweets


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-a2c189241d5b>", line 16, in <cell line: 0>
    sarcastic_tweets = [tweet for tweet in tweets_list if score_sarcasm_on_tweet(tweet["text"]) > 1]
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-18-a2c189241d5b>", line 16, in <listcomp>
    sarcastic_tweets = [tweet for tweet in tweets_list if score_sarcasm_on_tweet(tweet["text"]) > 1]
                                                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-18-a2c189241d5b>", line 2, in score_sarcasm_on_tweet
    context = client.features.inspect(
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/goodfire/api/features/client.py", line 763, in inspect
    return run_

TypeError: object of type 'NoneType' has no len()

## On-Demand RAG

If we see the user is asking about something that might need more data, e.g. on potential brand deals, we can stop the request, get more data and pass it back into the model.

For example, if the user asks about drinks, and we sponsor Coca Cola, we can stop the request, get RAG data on brand deals and pass it back into the model.

In [ ]:
consumerism_features = client.features.search("coca cola", model=variant, top_k=10)
print(consumerism_features)


In [ ]:
def get_rag_data(prompt):
    #this is where you would perform actual RAG search
    return "We have a brand deal with Coca Cola. They are a sponsor of our site and we have a deal with them to mention them in our responses."

def generate_response(prompt):

    variant.reset()
    variant.abort_when(consumerism_features[0] > 0.25)


    generated_tokens = ""
    try:
        for token in client.chat.completions.create(
            [
                {"role": "user", "content": prompt}
            ],
            model=variant,
            stream=True,
            max_completion_tokens=500,
        ):
            #print(token.choices[0].delta.content, end="")
            generated_tokens += token.choices[0].delta.content

        # If we never get to the brand deal, we'll just return the generated tokens
        print(generated_tokens)

    except Exception as e:
        print(e)
        rag_data = get_rag_data(prompt)
        print(generated_tokens)
        variant.reset()
        print("NEW TOKENS")
        for token in client.chat.completions.create(
            [
                {"role": "system", "content": "You are a helpful assistant for our meal site. You have access to the following information on brand deals:" + rag_data},
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": generated_tokens},
            ],
            model=variant,
            stream=True,
            max_completion_tokens=500,
        ):

            print(token.choices[0].delta.content, end="")

    return None

generate_response("What's are some good drinks to pair with pizza?")